In [ ]:
%pip install gurobipy
from gurobipy import *

ONHAND_INV = 5
FINAL_INV = 5
DEMAND = [10, 15, 30, 35, 25, 10]
PCOST = [1250, 1255, 1270, 1280, 1285, 1295]
ICOST = [60, 60, 60, 60, 60, 60]
PCAPA = [30, 30, 30, 30, 30, 30]
ICAPA = [10, 10, 10, 10, 10, 10]

n_months = 6

m=Model('Production Planning')

x = m.addVars(n_months, vtype = GRB.CONTINUOUS, name='x')
y = m.addVars(n_months, vtype = GRB.CONTINUOUS, name='y')


m.setObjective(quicksum(PCOST[i]*x[i]+ICOST[i]*y[i] for i in range(n_months)),GRB.MINIMIZE)

m.addConstr(ONHAND_INV + x[0] == DEMAND[0] + y[0])

m.addConstrs(y[i-1] + x[i] == DEMAND[i] + y[i] for i in range(1, n_months))

m.addConstrs(x[i] <= PCAPA[i] for i in range(n_months))

m.addConstrs(y[i] <= ICAPA[i] for i in range(n_months))

m.addConstr(y[5] == FINAL_INV)

m.write('production.lp')
m.optimize()

if m.SolCount > 0: #최적해 발견 flag
  m.printAttr('X')



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 38.1 MB/s eta 0:00:00
Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 19 rows, 12 columns and 30 nonzeros
Model fingerprint: 0xfd752fb5
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+01, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 4e+01]
Presolve removed 19 rows and 12 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6030000e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Opt

In [ ]:
%pip install gurobipy
from gurobipy import *

SOURCE = 'A'
TERMINAL = 'F'
nodes = {'A', 'B', 'C', 'D', 'E', 'F'}
arcs, cost=multidict({
    ('A', 'B') : 2,
    ('A', 'C') : 4,
    ('B', 'C') : 1,
    ('B', 'D') : 4,
    ('B', 'E') : 2,
    ('C', 'E') : 3,
    ('D', 'F') : 2,
    ('E', 'D') : 3,
    ('E', 'F') : 2,
})

m=Model('sp')

x = m.addVars(nodes, nodes, vtype = GRB.CONTINUOUS, name='x')

m.setObjective(quicksum(cost[(i, j)] * x[i,j] for i in nodes for j in nodes if (i, j) in arcs), GRB.MINIMIZE)

for i in nodes:
    if i == SOURCE:
        m.addConstr(quicksum(x[i, j] for j in nodes if (i, j) in arcs) - quicksum(x[j, i] for j in nodes if (j, i) in arcs) == 1)
    elif i == TERMINAL:
        m.addConstr(quicksum(x[i, j] for j in nodes if (i, j) in arcs) - quicksum(x[j, i] for j in nodes if (j, i) in arcs) == -1)
    else:
        m.addConstr(quicksum(x[i, j] for j in nodes if (i, j) in arcs) - quicksum(x[j, i] for j in nodes if (j, i) in arcs) == 0)

m.write('sp.lp')
m.optimize()
print(m.objVal)

if m.SolCount > 0: #최적해 발견 flag
  m.printAttr('X')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 6 rows, 36 columns and 18 nonzeros
Model fingerprint: 0xf81727d8
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 3 rows and 30 columns
Presolve time: 0.01s
Presolved: 3 rows, 6 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0000000e+00   0.000000e+00   0.000000e+00      0s
       0    6.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  6.000000000e+00
6.0

    Variable            X 
---------------------

In [ ]:
%pip install gurobipy
from gurobipy import *

n = 6
c= [2, 5, 6, 8, 4, 3]
M = 100

m = Model('0-1 Logical')

x = m.addVars(n, vtype = GRB.BINARY, name='x')
y= m.addVar(vtype = GRB.BINARY, name='y')

m.setObjective(quicksum(x[i]*c[i] for i in range(n)), GRB.MAXIMIZE)

m.addConstr(x.sum() >= 2)
m.addConstr(x.sum() <= 4)
m.addConstr(x[2] + x[4] <= 1)
m.addConstr(x[3] <= x[0])

m.addConstr(x[0]+x[1] <= 1 + M*y)
m.addConstr(x[4]+x[5] >= 1 - M*(1-y))

m.write('0-1.lp')

# 모델 최적화
m.optimize()

# display solution
if m.SolCount > 0: #최적해 발견 flag
  m.printAttr('X')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 6 rows, 7 columns and 22 nonzeros
Model fingerprint: 0x4ee34460
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 14.0000000
Presolve time: 0.00s
Presolved: 6 rows, 7 columns, 21 nonzeros
Variable types: 0 continuous, 7 integer (7 binary)
Found heuristic solution: objective 17.0000000

Root relaxation: objective 2.000000e+01, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  De